In [14]:
print("OK")

OK


In [1]:
%pwd

'd:\\Generative AI\\Langchain-Project\\Law-Assistant\\research'

In [15]:
import os
os.chdir("../")

In [16]:
%pwd

'd:\\Generative AI\\Langchain-Project\\Law-Assistant'

In [17]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
from langchain_community.document_loaders import CSVLoader, DirectoryLoader, PyPDFLoader
from langchain.docstore.document import Document
import os

# Load all CSV files first
def load_csv_documents(folder_path):
    csv_docs = []
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            loader = CSVLoader(file_path=os.path.join(folder_path, file))
            csv_docs.extend(loader.load())
    return csv_docs

# Load all PDFs after
def load_pdf_documents(folder_path):
    loader = DirectoryLoader(
        folder_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    return loader.load()

# Combine all documents: CSV first, then PDFs
def load_all_documents(folder_path):
    csv_docs = load_csv_documents(folder_path)
    pdf_docs = load_pdf_documents(folder_path)
    combined = csv_docs + pdf_docs  # CSV comes first
    print(f"Loaded {len(csv_docs)} CSV chunks and {len(pdf_docs)} PDF chunks")
    return combined


In [19]:
docs = load_all_documents("./")  # Root directory

Loaded 0 CSV chunks and 0 PDF chunks


In [20]:
print("Loaded document count:", len(docs))
if not docs:
    print("No documents were loaded.")


Loaded document count: 0
No documents were loaded.


In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embedding_model = download_hugging_face_embeddings()


In [22]:
query_result = embedding_model.embed_query("Hello world")
print("Length:", len(query_result))

Length: 384


In [23]:
# query_result

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [29]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name and parameters
index_name = "lawbot"

# Create index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


{
    "name": "lawbot",
    "metric": "cosine",
    "host": "lawbot-5kbkeko.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [30]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [31]:
import os
from langchain_community.document_loaders import PyPDFLoader, CSVLoader, DirectoryLoader

def load_all_documents(folder_path):
    csv_docs = []
    pdf_docs = []

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if file.endswith(".csv"):
            loader = CSVLoader(file_path=file_path)
            csv_docs.extend(loader.load())
        elif file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
            pdf_docs.extend(loader.load())

    print(f"Loaded {len(csv_docs)} CSV chunks and {len(pdf_docs)} PDF chunks")
    return csv_docs + pdf_docs


In [35]:
docs = load_all_documents("Data/")


Loaded 77104 CSV chunks and 109 PDF chunks


In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(f"Total final chunks: {len(chunks)}")


Total final chunks: 115242


In [37]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=chunks,
    index_name=index_name,
    embedding=embedding_model
)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 27 Jul 2025 20:22:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '94', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '4399', 'x-pinecone-request-id': '6542360143586607349', 'x-envoy-upstream-service-time': '18', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Request size 3MB exceeds the maximum supported size of 2MB","details":[]}


In [38]:
from langchain_pinecone import PineconeVectorStore

batch_size = 100  # or 200
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    PineconeVectorStore.from_documents(
        documents=batch,
        index_name=index_name,
        embedding=embedding_model
    )


KeyboardInterrupt: 